In [72]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense

In [78]:
def normalize_data(rows):
    max_columns = rows.max(axis=0)
    min_columns = rows.min(axis=0)
    norm_rows = np.empty((rows.size, 52))
    for i in range(rows.shape[0]):
        for j in range(rows.shape[1]):
            if ((max_columns[j]-min_columns[j])>0):
                norm_rows[j][i] = (rows[j][i] - min_columns[j])/(max_columns[j]-min_columns[j])
            else:
                norm_rows[j][i] = 0
    return norm_rows

normalized = normalize_data(data)
normalized

IndexError: index 51 is out of bounds for axis 0 with size 51

In [47]:
def import_data():
    data = pd.read_csv('data_kia.csv').values
    x_data = data[:, :51]
    y_data = data[:, 52]
    return x_data, y_data

data, labels = import_data()
data

(94380, 54)


array([[268.8, 0.0, 5.2, ..., -8.8, 0, -3.4],
       [243.2, 0.0, 6.1, ..., -0.2, 0, -3.6],
       [217.6, 0.0, 5.2, ..., -0.2, 0, -3.6],
       ...,
       [345.6, 0.0, 6.6, ..., 0.0, 0, -13.2],
       [332.8, 0.0, 5.7, ..., 0.0, 0, -13.3],
       [281.6, 0.0, 5.2, ..., -0.1, 0, -13.3]], dtype=object)

In [53]:
def onehot_labels(labels, num_labels=10):
    """
    Converts label arrya to 1-hot encoded vectors
    """

    result = []

    for l in labels:
        label_vector = [0] * num_labels
        index = ord(l) - ord('A')
        label_vector[index] = 1

        result.append(label_vector)

    return np.array(result)

y_data = onehot_labels(labels)

y_data

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [73]:
def package_data(data, window_size=16):
    result = []
    for i in range(len(data) - window_size):
        result.append(data[i:i + window_size])

    return result
x_data = tf.convert_to_tensor(package_data(data), dtype=tf.float64)
x_data[0].shape

2022-03-15 17:18:02.327056: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 616008192 exceeds 10% of free system memory.


TensorShape([16, 51])

In [74]:
def create_model(learning_rate=1e-4, window_size=16, num_features=51):
    model = Sequential()
    model.add(LSTM(160, input_shape=(window_size, num_features), return_sequences=True ))
    model.add(LSTM(200, ))

    model.add(Dense(10, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="categorical_crossentropy", metrics=['categorical_crossentropy'])

    return model

In [79]:
model = create_model()
model.fit(x_data, y_data[:94364])

 209/2949 [=>............................] - ETA: 2:07 - loss: 2.2211 - categorical_crossentropy: 2.2211

KeyboardInterrupt: 